<a href="https://colab.research.google.com/github/j84m9/j84m9/blob/main/Kaggle_Housing_Prices_Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture

!pip install sklearn
!pip install optuna
!pip install pandas

In [3]:
from google.colab import files

uploaded = files.upload()

Saving train.csv to train.csv


In [4]:
import pandas as pd
import io

train = pd.read_csv(io.BytesIO(uploaded["train.csv"])).drop(columns = ['Id'])
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
#numerical column names
num_cols = [i for i in train.columns if train[i].dtype != 'O']

#Obj_cols
obj_cols = [i for i in train.columns if train[i].dtype == 'O']

In [6]:
# num_w_missing = []
# missing_dict = {}
# for i in num_cols:
#   if train[i].isnull().sum() > 0:
#     num_w_missing.append(i)
#     missing_dict[i] = train[i].isnull().sum()

# print(num_w_missing)

#Filling missing numerical columnvalues
train['LotFrontage'] = train['LotFrontage'].fillna(0)

train['MasVnrArea'] = train['MasVnrArea'].fillna(0)

train['GarageYrBlt'] = train['GarageYrBlt'].fillna(-1)

In [7]:
garage_vars = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
for i in garage_vars:
    if i == 'GarageYrBlt':
        train[i].fillna(0, inplace = True)
    else:
        train[i].fillna('No', inplace = True)
#------------------------------------------------------------------------------------------------
bsmt_vars = ['BsmtQual','BsmtCond']
for i in bsmt_vars:
    train[i].fillna('No', inplace = True)

In [8]:
bsmt_garage_ordinal_encoding = {'Ex': 5,'Gd': 4,'TA': 3,'Fa': 2,'Po': 1, 'No': 0}
for i in ['GarageQual', 'GarageCond', 'BsmtQual', 'BsmtCond']:
    train[i] = train[i].map(bsmt_garage_ordinal_encoding)

In [9]:
cols = num_cols + garage_vars + bsmt_vars + ['SaleCondition']
df = train[cols]
df.drop(columns = ['GarageType', 'GarageFinish'], inplace = True)
df.head()

<ipython-input-9-022e95dc1707>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = ['GarageType', 'GarageFinish'], inplace = True)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,PoolArea,MiscVal,MoSold,YrSold,SalePrice,GarageQual,GarageCond,BsmtQual,BsmtCond,SaleCondition
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,2,2008,208500,3,3,4,3,Normal
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,5,2007,181500,3,3,4,3,Normal
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,9,2008,223500,3,3,4,3,Normal
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,2,2006,140000,3,3,3,4,Abnorml
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,12,2008,250000,3,3,4,3,Normal


In [10]:
from sklearn.preprocessing import OneHotEncoder

#Create an Instance of OneHotEncoder
encoder = OneHotEncoder(sparse = False)

#Fit the encoder and transform the data
OHE = encoder.fit_transform(df[['SaleCondition']])

#Create a DataFrame with the OneHotEncoded Columns
OHE_df = pd.DataFrame(OHE, columns = encoder.get_feature_names_out())

#Concatenate tje one-hot encoded DataFrame with the original data
res = pd.concat([df, OHE_df], axis = 1)
res = res.drop(columns = ['SaleCondition'])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [11]:
from sklearn.model_selection import train_test_split
X = res[[i for i in res.columns if i != 'SalePrice']]
y = res['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 1234)

In [13]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import optuna
import time

ST = time.time()
def objective(trial):
    #Define hyperparameters to optimize
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
    n_estimators = trial.suggest_int("n_estimators", 100, 1000)
    max_depth = trial.suggest_int("max_depth", 1, 30)

    #Create and train model using the hyperparameters
    model = XGBRegressor(learning_rate = learning_rate, n_estimators = n_estimators, max_depth = max_depth, nthread = -1)
    model.fit(X_train, y_train)

    #Calculate metric to optimize (e.g. accuracy, RMSE, etc.)
    predictions = model.predict(X_test)
    score = mean_absolute_error(y_test, predictions)

    return score

study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials = 1000)# , verbose=0)
ET = time.time()

[I 2023-09-17 15:05:49,856] A new study created in memory with name: no-name-8e745200-aacd-4d55-ab6d-a465e0e13b88
[I 2023-09-17 15:06:01,896] Trial 0 finished with value: 16678.84652183219 and parameters: {'learning_rate': 0.04071845004042694, 'n_estimators': 741, 'max_depth': 24}. Best is trial 0 with value: 16678.84652183219.
[I 2023-09-17 15:06:03,676] Trial 1 finished with value: 16546.180169092466 and parameters: {'learning_rate': 0.07681401695854716, 'n_estimators': 242, 'max_depth': 11}. Best is trial 1 with value: 16546.180169092466.
[I 2023-09-17 15:06:10,068] Trial 2 finished with value: 16602.184460616438 and parameters: {'learning_rate': 0.09971358679359618, 'n_estimators': 597, 'max_depth': 13}. Best is trial 1 with value: 16546.180169092466.
[I 2023-09-17 15:06:10,941] Trial 3 finished with value: 16373.920451626713 and parameters: {'learning_rate': 0.05143605466423805, 'n_estimators': 244, 'max_depth': 6}. Best is trial 3 with value: 16373.920451626713.
[I 2023-09-17 15:

In [15]:
print((ET - ST)/60)

35.82568538983663


In [17]:
best_params = study.best_params
best_score = study.best_value

print(f"The optimal parameters are: {best_params}")
print(f"The best MAE was: {best_score}")

The optimal parameters are: {'learning_rate': 0.09324404142859685, 'n_estimators': 824, 'max_depth': 2}
The best MAE was: 14457.794188784246


In [19]:
best_params = {'learning_rate': 0.09324404142859685, 'n_estimators': 824, 'max_depth': 2}
best_model = XGBRegressor(n_estimators = best_params["n_estimators"], max_depth = best_params['max_depth'], learning_rate = best_params['learning_rate'])
best_model.fit(X, y)


# y_pred = best_model.predict(X_test)
# mean_absolute_error(y_test, y_pred)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.09324404142859685,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=824, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [20]:
from google.colab import files

uploaded = files.upload()

Saving test.csv to test.csv


In [23]:
test = pd.read_csv(io.BytesIO(uploaded["test.csv"])).drop(columns = ['Id'])
test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [24]:
#test = pd.read_csv("test.csv").drop(columns = ['Id'])

#numerical column names
num_cols = [i for i in test.columns if test[i].dtype != 'O']

#Obj_cols
obj_cols = [i for i in test.columns if test[i].dtype == 'O']


#Filling missing values for numerical columns
test['LotFrontage'] = test['LotFrontage'].fillna(0)

test['MasVnrArea'] = test['MasVnrArea'].fillna(0)

test['GarageYrBlt'] = test['GarageYrBlt'].fillna(-1)

#Filling Missing Garage Variables:
#garage_vars = ['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond']
garage_vars = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
for i in garage_vars:
    if i == 'GarageYrBlt':
        test[i].fillna(0, inplace = True)
    else:
        test[i].fillna('No', inplace = True)

# Filling Missing Basement Variables
bsmt_vars = ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']

for i in bsmt_vars:
    test[i].fillna('No', inplace = True)


cols = num_cols + garage_vars + bsmt_vars + ['SaleCondition']
test_df = test[cols]

#train.PoolQC.fillna('No', inplace = True)
bsmt_garage_ordinal_encoding = {'Ex': 5,'Gd': 4,'TA': 3,'Fa': 2,'Po': 1, 'No': 0}

for i in ['GarageQual', 'GarageCond', 'BsmtQual', 'BsmtCond']:
    test_df[i] = test_df[i].map(bsmt_garage_ordinal_encoding)

drop_cols = ['GarageType', 'GarageFinish', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
test_df.drop(columns = drop_cols, inplace = True)

#df.head()

OHE = encoder.transform(test_df[['SaleCondition']])
OHE_df = pd.DataFrame(OHE, columns = encoder.get_feature_names_out())

res = pd.concat([test_df, OHE_df], axis = 1)
res = res.drop(columns = ['SaleCondition'])

<ipython-input-24-8ed3b087edab>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[i] = test_df[i].map(bsmt_garage_ordinal_encoding)
<ipython-input-24-8ed3b087edab>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[i] = test_df[i].map(bsmt_garage_ordinal_encoding)
<ipython-input-24-8ed3b087edab>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [25]:
test_preds = best_model.predict(res)
out = pd.DataFrame(test_preds)
out.columns = ['SalePrice']
test = pd.read_csv('test.csv')
res2 = pd.concat([test['Id'],out], axis = 1)

In [26]:
res2.to_csv('colab_test.csv', index = False)

files.download('colab_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>